### Plugin setup for ImJoy

In [ ]:
from imjoy import api
import zarr

def encode_zarr_store(zobj):
    path_prefix = f"{zobj.path}/" if zobj.path else ""
    def getItem(key):
        return zobj.store[path_prefix + key]

    def setItem(key, value):
        zobj.store[path_prefix + key] = value

    def containsItem(key):
        if path_prefix + key in zobj.store:
            return True

    return {
        "_rintf": True,
        "_rtype": 'zarr-array' if isinstance(zobj, zarr.Array) else 'zarr-group',
        "getItem": getItem,
        "setItem": setItem,
        "containsItem": containsItem,
    }

api.registerCodec({'name': 'zarr-array', 'type': zarr.Array, "encoder": encode_zarr_store})
api.registerCodec({'name': 'zarr-group', 'type': zarr.Group, "encoder": encode_zarr_store})
             
class VivPlugin:
    def __init__(self, images, view_state=None):
        if not isinstance(images, list):
            images = [images]
        self.images = images
        self.view_state = view_state
        
    async def setup(self):
        pass
        
    async def run(self, ctx):
        viewer = await api.createWindow(
            type="vitessce-image-viewer-plugin", 
            src="https://epic-poitras-2b80af.netlify.app/"
#             src="http://localhost:8080"
        )
        if self.view_state:
            await viewer.set_view_state(self.view_state)
        for img in self.images:
            await viewer.add_image(img)
        
def run_viv(images, view_state=None):
    api.export(VivPlugin(images, view_state))

### Create a multiscale OME-Zarr layer

In [ ]:
z_grp = zarr.open('astronaut.zarr')

image = {
    "source": z_grp, # z_group is for an ome-zarr that contains the multiscale metadata
    "opacity": 1, # default is 1
    "colormap": None, # default, can be "viridis", "turbo", see viv.vitessce.io for all options
}

### Create a generic array layer

In [ ]:
import numpy as np

# Need to add extra dim and cast to f32
arr = np.random.rand(1024, 1024)[None,:,:].astype('f4')
                             
noise = {
    "source": zarr.array(arr), 
    "dimensions": "cyx", # if not OME-Zarr, need to label dimensions for array
    "layers": [ # Manually define list of 2D panes to additively blend
        { 
            "contrast_limits": [0, 1], # required 
            "selection": { "c": 0 }, # required,
            "color": "#FFFFFF", # optional
            "label": "noise layer", # options, default channel_x
        },
    ],
    "opacity": 0.5,
}

### Run plugin and view layers

In [ ]:
run_viv([image, noise])

In [ ]:
run_viv([image, noise], view_state={ "zoom": -3, "target": [512, 512, 0] })

In [ ]:
z = zarr.open('data/LuCa-7Color/data.zarr/')
img = { "source": z.get('0') }
run_viv(img)